In [1]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from crewai import Agent
from crewai_tools import SerperDevTool
from crewai import Task
from crewai import Crew, Process

In [2]:
# Load environment variables from .env file
load_dotenv()

True

In [3]:
# Access the environment variables
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GROQ_API_BASE = os.getenv("GROQ_API_BASE") 
GROQ_MODEL = os.getenv("GROQ_MODEL", "groq/llama-3.3-70b-versatile")


sender_email = os.getenv("SENDER_EMAIL")
sender_password = os.getenv("SENDER_PASSWORD")
recipient_email = os.getenv("RECIPIENT_EMAIL")

In [4]:
search_tool = SerperDevTool()

In [5]:
import os
from crewai import LLM

groq_llm = LLM(
        model = GROQ_MODEL,
        api_key = GROQ_API_KEY,
        base_url = GROQ_API_BASE,
        temperature=0.3, 
        max_tokens=4096)

llm = groq_llm

In [6]:
# Creating a senior researcher agent with memory and verbose mode
topic_researcher = Agent(
    role="Researcher and Report Writer",
    goal="Research and produce a clear, structured report in {topic}",
    verbose=True,
    memory=False,
    backstory=(
       "An experienced AI analyst with deep research and writing capabilities."
       "Known for producing concise, factual reports on complex topics."
    ),
    tools=[search_tool],
    allow_delegation=True,
    llm= llm,
)

In [7]:
#critic = Agent(
#    role="Compliance Critic",
#   goal="Ensure that research output adheres to legal, ethical, and factual standards",
#    backstory="Responsible for analyzing content for misinformation, copyright issues, and ethical violations.",
 #   verbose=True,
  #  memory = True,
   # #tools=[search_tool],
    #allow_delegation=True, 
    #llm= llm,
#) 

In [8]:
research_task = Task(
    description= "Conduct web research on: '{topic}' and gather relevant data."
                 "and produce a well-structured report with an introduction, findings, and conclusion.",
    expected_output="A concise research summary report with key points and source links.",
    tools=[search_tool],
    agent=topic_researcher,
    output_file= "final_report.md"
)

In [9]:
#critic_task = Task( 
    #description="Evaluate the summary for compliance with legal, ethical, and factual standards. "
                #"Ensure no false claims, offensive language, or unverified info is present.",
    #expected_output="A compliance-checked version of the summary with necessary corrections.",
    #agent=critic,
    #tools=[search_tool],
    #context=[research_task],
    #output_file= "final_compliance_checked.md" )

In [10]:
# Task: Reporter revises based on critic feedback
#revision_task = Task(
    #description="Revise the report using the feedback provided by the critic. Apply all suggested corrections while maintaining clarity and accuracy.",
   # expected_output="A final report that incorporates all feedback and is ready for publishing.",
    #agent=topic_researcher,
    #tools=[search_tool],
    #context=[research_task],  # feeds critic’s feedback back to reporter
    #output_file="final-report.md"
#)

In [11]:
# Forming the crew with enhanced configurations
crew = Crew(
    agents=[topic_researcher],
    tasks=[research_task],
    process=Process.sequential 
)

# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'The role of AI in improving food security in Africa'})
print(result)

# Agent: Researcher and Report Writer
## Task: Conduct web research on: 'The role of AI in improving food security in Africa' and gather relevant data.and produce a well-structured report with an introduction, findings, and conclusion.


# Agent: Researcher and Report Writer
## Thought: Thought: To conduct web research on 'The role of AI in improving food security in Africa', I should start by searching for relevant information on the internet. This will help me gather data and produce a well-structured report with an introduction, findings, and conclusion.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"The role of AI in improving food security in Africa\"}"
## Tool Output: 
{'searchParameters': {'q': 'The role of AI in improving food security in Africa', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Is Artificial Intelligence the future of farming? Exploring opportunities ...', 'link': 'https://blogs.worldbank.org/en/agf

In [12]:
# Add Name and Project Title to PDF 
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from datetime import datetime
import os

def generate_pdf_report(content, filename="final_report.pdf", folder="reports",
                        author_name="Osita Wisdom Chinedu",
                        project_title="AI Research on Food Security in Africa"):

    if not os.path.exists(folder):
        os.makedirs(folder)

    filepath = os.path.join(folder, filename)
    doc = SimpleDocTemplate(filepath, pagesize=A4,
                            rightMargin=50, leftMargin=50,
                            topMargin=50, bottomMargin=50)

    styles = getSampleStyleSheet()

    # Rename custom styles to avoid name conflicts
    styles.add(ParagraphStyle(name='Justify', alignment=4, leading=14, fontSize=11))
    styles.add(ParagraphStyle(name='CustomTitle', parent=styles['Heading1'], alignment=1, fontSize=16, spaceAfter=12))
    styles.add(ParagraphStyle(name='CustomSubtitle', alignment=1, fontSize=12, spaceAfter=6))

    flowables = []

    flowables.append(Paragraph(project_title, styles['CustomTitle']))
    flowables.append(Paragraph(f"Author: {author_name}", styles['CustomSubtitle']))
    flowables.append(Paragraph(f"Date: {datetime.now().strftime('%B %d, %Y')}", styles['CustomSubtitle']))
    flowables.append(Spacer(1, 12))

    for line in content.strip().split("\n"):
        if line.strip() == "":
            flowables.append(Spacer(1, 10))
        else:
            flowables.append(Paragraph(line.strip(), styles['Justify']))

    doc.build(flowables)
    print(f"✅ PDF report generated at: {filepath}")
    return filepath


In [13]:
# Add Auto-Email Feature 
import yagmail
import os

def send_email_with_attachment(receiver_email, subject, body, attachment_path):

    sender_email = os.getenv("SENDER_EMAIL")
    sender_password = os.getenv("SENDER_PASSWORD")
   

    if not sender_email or not sender_password:
        raise ValueError("Please set SENDER_EMAIL and SENDER_PASSWORD in your environment variables.")

    yag = yagmail.SMTP(sender_email, sender_password)
    yag.send(to=receiver_email, subject=subject, contents=body, attachments=attachment_path)

    print(f"\n📧 Email sent to {receiver_email} with the PDF attachment.")


In [14]:
# GENERATE PDF
pdf_path = generate_pdf_report(
    content=str(result),
    filename="AI_Research_Report.pdf",
    author_name="Osita Wisdom Chinedu",
    project_title="AI Research on Food Security in Africa"
)

✅ PDF report generated at: reports\AI_Research_Report.pdf


In [15]:
# SEND EMAIL
send_email_with_attachment(
    receiver_email=recipient_email,
    subject="AI Research Report",
    body="Attached is your finalized research report.",
    attachment_path=pdf_path
)


📧 Email sent to ositawisdomc@gmail.com with the PDF attachment.
